# 🧪 NLP Lab: N-Gram Language Modeling and Naïve Bayes Text Classification

This notebook is a step-by-step lab for students. It contains *theory*, *formulas*, explanations, and runnable *code cells* for each part.

Sections:

1. Setup
2. N-gram language modeling (unigram/bigram/trigram)
3. Add-1 (Laplace) smoothing and sentence probability
4. Perplexity calculation
5. Naïve Bayes text classification (with negation & lexicon features)
6. Exercises and deliverables


## 🎯 Learning Objectives

- Build unigram, bigram, and trigram language models from text.
- Implement additive (Laplace) smoothing and compute sentence probabilities.
- Compute perplexity as an intrinsic evaluation metric.
- Use Naïve Bayes for sentiment classification on a small corpus.
- Explore negation handling and lexicon feature augmentation.


---

## 🔧 Setup

Run the cell below to install required packages and import libraries. If you're using Google Colab the `!pip install` will be fast. On local machines, ensure you have network access to download NLTK corpora.


In [ ]:
# !pip install -q nltk pandas numpy scikit-learn matplotlib nbformat

In [2]:
import nltk
from nltk.util import ngrams
from nltk.corpus import reuters, movie_reviews
from collections import Counter, defaultdict
import math, random, numpy as np
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize, word_tokenize

# Download corpora (only the first time)
# nltk.download('reuters')
# nltk.download('movie_reviews')
# nltk.download('punkt')
# nltk.download('punkt_tab') 


---

## Part 1 — Tokenization & N-gram Counts

**Goal:** create unigram, bigram, and trigram counts from a corpus (Reuters `acq` category).

**Notation / formulas**:

- Let \(w_i\) be the word at position \(i\).
- Unigram probability (maximum likelihood estimate):

\[ P(w) = \frac{C(w)}{N} \]

where \(C(w)\) is the count of \(w\) and \(N\) is total tokens.

- Bigram conditional probability (MLE):

\[ P(w_i \mid w_{i-1}) = \frac{C(w_{i-1}, w_i)}{C(w_{i-1})} \]

We'll compute counts with NLTK and `collections.Counter`.


In [4]:
# Load sentences from Reuters (acq category)
text = " ".join(reuters.words(categories='acq')[:5000])
docs = [word_tokenize(sent) for sent in sent_tokenize(text)]
# Flatten tokens and add sentence boundary tokens
tokens = ['<s>'] + [w.lower() for sent in docs for w in sent] + ['</s>']

# Helper to build n-gram counts
def build_ngram_counts(tokens, n):
    return Counter(ngrams(tokens, n))

unigram_counts = build_ngram_counts(tokens, 1)
bigram_counts = build_ngram_counts(tokens, 2)
trigram_counts = build_ngram_counts(tokens, 3)

V = len(set(tokens))  # vocabulary size

print('Vocabulary size V =', V)
print('Sample unigram counts (top 10):', unigram_counts.most_common(10))
print('Sample bigram counts (top 10):', bigram_counts.most_common(10))


Vocabulary size V = 1319
Sample unigram counts (top 10): [(('.',), 264), (('the',), 223), ((',',), 183), (('of',), 124), (('to',), 95), (('said',), 93), (('a',), 90), (('and',), 75), (('in',), 74), (('s',), 67)]
Sample bigram counts (top 10): [(("'", 's'), 50), (('&', 'lt'), 45), (('lt', ';'), 45), (('of', 'the'), 24), (('.', 'the'), 23), (('said', '.'), 19), (('said', 'it'), 19), (('.', '``'), 16), (('.', 's'), 16), ((',', 'the'), 16)]


---

## Part 2 — Laplace (Add-1) Smoothing

**Problem:** if a bigram \((w_{i-1}, w_i)\) never appears in training, \(C(w_{i-1}, w_i)=0\) and the MLE gives zero probability.

**Laplace (Add-1) smoothing** adds 1 to every bigram count to avoid zeros:

\[ P_{Laplace}(w_i \mid w_{i-1}) = \frac{C(w_{i-1}, w_i) + 1}{C(w_{i-1}) + V} \]

where \(V\) is the vocabulary size (number of distinct tokens).

This guarantees every possible next word has non-zero probability.


In [5]:
# Define smoothed bigram probability

def bigram_prob(w1, w2, bigram_counts=bigram_counts, unigram_counts=unigram_counts, V=V):
    return (bigram_counts[(w1, w2)] + 1) / (unigram_counts[(w1,)] + V)

# Example: show probabilities for some bigrams
examples = [('the', 'company'), ('company', 'made'), ('made', 'a'), ('a', 'profit'), ('profit','</s>')]
for a, b in examples:
    print(f"P({b}|{a}) =", bigram_prob(a, b))


P(company|the) = 0.009727626459143969
P(made|company) = 0.0007446016381236039
P(a|made) = 0.0015117157974300832
P(profit|a) = 0.0007097232079489
P(</s>|profit) = 0.000757002271006813


### Sentence probability (chain rule using bigrams)

Given a sentence \(w_1, w_2, \dots, w_n\) with start token `<s>` and end token `</s>`,

\[ P(s) = \prod_{i=1}^{n} P(w_i \mid w_{i-1}) \]

We compute this using the smoothed bigram probabilities.


In [10]:
import nltk

def sentence_prob(sentence, tokenizer=nltk.word_tokenize):
    sent = ['<s>'] + tokenizer(sentence.lower()) + ['</s>']
    log_prob = 0.0
    for i in range(1, len(sent)):
        p = bigram_prob(sent[i-1], sent[i])
        log_prob += math.log(p)
    return math.exp(log_prob), log_prob  # return probability and log-prob

s1 = 'the company made a profit'
s2 = 'profit company the made'

p1, lp1 = sentence_prob(s1)
p2, lp2 = sentence_prob(s2)

print(f"Sentence: {s1}\n  Prob = {p1:.3e}, log-prob = {lp1:.3f}")
print(f"Sentence: {s2}\n  Prob = {p2:.3e}, log-prob = {lp2:.3f}")


Sentence: the company made a profit
  Prob = 4.457e-18, log-prob = -39.952
Sentence: profit company the made
  Prob = 2.093e-16, log-prob = -36.103


**Interpretation:** the grammatical sentence should have a higher probability (less negative log-prob). We often work with log-probabilities to avoid numeric underflow.

---

## Part 3 — Perplexity

Perplexity measures how well a probability model predicts unseen text. For a test set of tokens of length \(N\) with log-probability \(\log P(W)\):

\[ perplexity = \exp\left(-\frac{1}{N} \log P(W) \right) \]

Lower perplexity indicates a better model.


In [7]:
def perplexity(test_sents):
    # test_sents: list of tokenized sentences (tokens already)
    N = sum(len(s) + 1 for s in test_sents)  # +1 for </s>
    log_prob = 0.0
    for s in test_sents:
        sent = ['<s>'] + s + ['</s>']
        for i in range(1, len(sent)):
            log_prob += math.log(bigram_prob(sent[i-1], sent[i]))
    return math.exp(-log_prob / N)

# Build some held-out test sentences (use last 50 sentences from Reuters)
test_docs = [ [w.lower() for w in sent] for sent in reuters.sents(categories='acq')[2000:2050] ]
print('Perplexity on 50 held-out Reuters sentences:', perplexity(test_docs))


Perplexity on 50 held-out Reuters sentences: 866.6795836049256


### Exercise: Why smoothing improves perplexity?

Because smoothing assigns non-zero probability to previously unseen n-grams, it avoids the model's total probability going to zero on test data, which would lead to infinite perplexity. Proper smoothing redistributes probability mass and typically lowers perplexity on unseen data.

---

## Part 4 — Naïve Bayes Text Classification

**Goal:** classify movie reviews (positive/negative) using bag-of-words Naïve Bayes with Laplace smoothing.

**Model:**

We want \(P(c \mid d) \propto P(c) \prod_{i} P(w_i \mid c)\),

where we estimate:

\[ P(w \mid c) = \frac{C(w,c) + 1}{\sum_{w'} C(w',c) + V} \]

and use log probabilities for numerical stability.


In [ ]:
# Prepare movie reviews dataset
docs = [(list(movie_reviews.words(fileid)), category)
        for category in movie_reviews.categories()
        for fileid in movie_reviews.fileids(category)]

train, test = train_test_split(docs, test_size=0.2, random_state=42)

# Build class-specific word counts
word_counts = {'pos': Counter(), 'neg': Counter()}
for words, label in train:
    word_counts[label].update(w.lower() for w in words)

V_nb = len(set(word_counts['pos']) | set(word_counts['neg']))
total_counts = {c: sum(word_counts[c].values()) for c in ['pos','neg']}

print('V (vocab for NB) =', V_nb)
print('Total token counts per class:', total_counts)


In [ ]:
def P_word_given_class(word, c):
    return (word_counts[c][word] + 1) / (total_counts[c] + V_nb)

def P_class_given_doc(words):
    # returns the predicted class using log-probs
    log_probs = {}
    for c in ['pos', 'neg']:
        log_prob = math.log(0.5)  # assume equal priors
        for w in words:
            log_prob += math.log(P_word_given_class(w.lower(), c))
        log_probs[c] = log_prob
    return max(log_probs, key=log_probs.get)

# Evaluate on a subset of the test set
correct = 0
for words, label in test[:200]:
    pred = P_class_given_doc(words)
    correct += (pred == label)

print('Accuracy (first 200 test docs):', correct / 200)


### Negation Handling & Lexicon Features

Naïve Bayes can misclassify phrases like "not good" because it treats words independently. A common trick is to prepend a `NOT_` marker to words after a negation word until the next punctuation. Example:

`I do not like this movie` → tokens: `I do not NOT_like NOT_this NOT_movie`

Lexicon features add a token if any known positive/negative words appear (`LEX_POS`, `LEX_NEG`) which gives the classifier a prior sentiment signal.


In [1]:
# Simple negation handling and lexicon augmentation
negation_words = {'not', "n't", 'never', 'no'}
positive_lexicon = {'good', 'excellent', 'great', 'amazing', 'love', 'nice'}
negative_lexicon = {'bad', 'awful', 'terrible', 'hate', 'boring', 'poor'}

import string

def apply_negation_and_lexicon(words):
    out = []
    negate = False
    for w in words:
        lw = w.lower()
        # break negation on sentence punctuation
        if any(p in lw for p in ['.', '!', '?']):
            negate = False
        if lw in negation_words:
            negate = True
            out.append(lw)
            continue
        if negate and lw.isalpha():
            out.append('NOT_' + lw)
        else:
            out.append(lw)
    # lexicon tokens
    if any(w in positive_lexicon for w in [x.lower().lstrip('not_') for x in out]):
        out.append('LEX_POS')
    if any(w in negative_lexicon for w in [x.lower().lstrip('not_') for x in out]):
        out.append('LEX_NEG')
    return out

# Try on a sample sentence
sample = "I did not like this movie, it was not good at all."
print('Original tokens:', sample.split())
print('Augmented tokens:', apply_negation_and_lexicon(sample.split()))


Original tokens: ['I', 'did', 'not', 'like', 'this', 'movie,', 'it', 'was', 'not', 'good', 'at', 'all.']
Augmented tokens: ['i', 'did', 'not', 'NOT_like', 'NOT_this', 'movie,', 'NOT_it', 'NOT_was', 'not', 'NOT_good', 'NOT_at', 'all.', 'LEX_POS']


In [12]:
# Evaluate NB with negation + lexicon on a small subset
# Build augmented training counts
word_counts_aug = {'pos': Counter(), 'neg': Counter()}
for words, label in train:
    aug = apply_negation_and_lexicon(words)
    word_counts_aug[label].update(aug)

V_aug = len(set(word_counts_aug['pos']) | set(word_counts_aug['neg']))

total_aug = {c: sum(word_counts_aug[c].values()) for c in ['pos','neg']}

def P_word_given_class_aug(word, c):
    return (word_counts_aug[c][word] + 1) / (total_aug[c] + V_aug)

def P_class_given_doc_aug(words):
    log_probs = {}
    for c in ['pos','neg']:
        log_prob = math.log(0.5)
        for w in apply_negation_and_lexicon(words):
            log_prob += math.log(P_word_given_class_aug(w, c))
        log_probs[c] = log_prob
    return max(log_probs, key=log_probs.get)

# Test on subset
correct = 0
for words, label in test[:200]:
    pred = P_class_given_doc_aug(words)
    correct += (pred == label)
print('Accuracy with negation+lexicon (first 200 test docs):', correct / 200)


NameError: name 'train' is not defined

## Exercises

1. Run Exercise 1 (sentence probs). Report which sentence has higher probability and why.
2. Compute perplexity on a held-out set and compare vanilla bigram vs add-1 smoothed bigram (you can simulate zero-probability by removing some bigrams from the counts).
3. Train Naïve Bayes with and without stopword removal; compare accuracies.
4. Try different lexicon sizes (small vs large) and observe impact.
5. Implement interpolation between unigrams, bigrams, and trigrams using held-out data to tune weights.

## Deliverables

- A filled notebook with outputs and short comments on observations.
- A 1-page reflection comparing statistical (n-gram / NB) vs lexical approaches.

---

*End of lab.*
